In [1]:
import gzip
import numpy as np

In [2]:
from sklearn.ensemble import RandomForestClassifier

#tree_depth_size = [10,15,25]
#metoda koja trenira Random Forest i vraća šumu (listu stabla)
def createRandomForest(X,y,depth_size = 25):
    clf = RandomForestClassifier(
            n_estimators=100, max_features="sqrt", max_depth=depth_size,
            min_samples_split=5)

    clf.fit(X,y)
    forest = clf.estimators_
    return forest

In [3]:
def mapToIndexesLeaves(tree):
    mapa = {}
    counter = 0
    mapa,counter = rekurzivno(tree,mapa,counter,0)
    return mapa

def rekurzivno(tree, mapa,counter,node):
        if tree.children_left[node] == -1:
            mapa[node] = counter
            counter +=1
            return mapa, counter
        mapa, counter = rekurzivno(tree, mapa, counter, tree.children_left[node])
        mapa, counter = rekurzivno(tree, mapa, counter, tree.children_right[node])
        return mapa, counter


In [4]:
from scipy.sparse import csr_matrix

def leafIndexesOfLeaves(treeDecision,mapa, X, offset = 0):
    return np.array([mapa[x] + offset for x in treeDecision.apply(X)])

def createFI(X, forest, forestMap):
    col = np.array([])
    row = np.array([])
    offset = 0
    for i,(f,mapa) in enumerate(zip(forest,forestMap)):
        indexes = leafIndexesOfLeaves(f,mapa, X,offset=offset)
        col = np.concatenate((col,indexes))
        row = np.concatenate((row,np.arange(0,X.shape[0])))
        offset += len(mapa)
    FI = csr_matrix(
        (np.ones(col.shape,dtype=np.int8), (row, col)), shape=(X.shape[0],offset))
    return FI

In [5]:
from liblinear.liblinearutil import *

def leafVectorOptimizator(X,y,s,c):
    prob  = problem(y, X)
    param = parameter('-s '+str(s)+' -c '+str(c)+' -q')
    m = train(prob, param)
    [W, _] = m.get_decfun()
    return W,m
def getScore(X,y,m):
    _,score,_ = predict(y, X, m)
    return score

In [6]:
def getNeighbors(forest, forestMap,W):
    neighbors = []
    offset = 0
    for i,(mapa,f) in enumerate(zip(forestMap,forest)):
        prev_node_index = None
        prev_leaf_index = None
        for (node_index, leaf_index) in mapa.items():
            if prev_node_index is None:
                prev_node_index = node_index
                prev_leaf_index = leaf_index
                continue
            
            if f.tree_.children_right[prev_node_index - 1] == node_index:
                singf = abs(W[offset + prev_leaf_index]) + abs (W[offset + leaf_index])
                neighbors.append([singf,i,prev_node_index-1])
                prev_node_index = None
                prev_leaf_index = None
                continue
            prev_node_index = node_index
            prev_leaf_index = leaf_index
        offset += len(mapa)
    return neighbors

In [7]:
from sklearn.tree._tree import TREE_LEAF

def pruneTrees(neighbors, forest,  pruneSize):
    neighbors.sort(key = lambda x : x[0])
    for _,f_index,parent in neighbors[:pruneSize]:
        forest[f_index].tree_.children_left[parent] = TREE_LEAF
        forest[f_index].tree_.children_right[parent] = TREE_LEAF

In [8]:
problemTypes = {
    "classifation":2,
    "regression":11 
}
import math
import copy

def refinedA(X_train,y_train, X_test, y_test,problemType,
    iteration = 10, c = 1,depth_size_index = 2):
    s = problemTypes[problemType]
    bestScore = None
    bestM = None
    tree_depth_size = [10,15,25]
    forest = createRandomForest(X_train,y_train,tree_depth_size[depth_size_index])
    counter = 0
    while True:
        forestMap = [mapToIndexesLeaves(f.tree_) for f in forest]
        FI_train = createFI(X_train,forest,forestMap)
        FI_test = createFI(X_test,forest,forestMap)
        W, m = leafVectorOptimizator(FI_train,y_train, s, c)
        score = getScore(FI_test,y_test, m)
        counter += 1
        if s == 2:
            score = score[0] / 100
            print("iteracija: " + str(counter) + ", error: " + 
            str(round(1- score,4)) +", len(W): "+ str(len(W)))
            if bestScore is None or score > bestScore:
                bestScore = score
                bestM = m

            
        else:
            RMSE = math.sqrt(score[1])
            print("iteracija: " + str(counter) + ", Root Mean Square Error(RMSE): " + 
            str(round(RMSE,4)) +", len(W): "+ str(len(W)))
            if bestScore is None or RMSE < bestScore:
                bestScore = copy.deepcopy(RMSE)
                bestM = m
                
        if counter > iteration:
                break
        
        neighbors = getNeighbors(forest, forestMap, W)
        pruneTrees(neighbors, forest, int(len(W) * 0.1))
    return bestM, bestScore

#Treniramo model i smanjujemo šumu dok ne greška ne počne rasti i dosegne target_error
def refinedE(X_train,y_train, X_test, y_test,problemType,
    target_error, c = 1,depth_size_index = 2):
    s = problemTypes[problemType]
    bestScore = None

    bestM = None
    tree_depth_size = [10,15,25]
    forest = createRandomForest(X_train,y_train,tree_depth_size[depth_size_index])
    size = 1
    counter = 0
    while True:
        forestMap = [mapToIndexesLeaves(f.tree_) for f in forest]
        FI_train = createFI(X_train,forest,forestMap)
        FI_test = createFI(X_test,forest,forestMap)
        W, m = leafVectorOptimizator(FI_train,y_train, s, c)
        score = getScore(FI_test,y_test, m)
        counter += 1
        if s == 2:
            score = score[0] / 100
            print("iteracija: " + str(counter) + ", error: " + 
            str(round(1 - score,4)) +", size: "+ str(round(size,4)))
            if bestScore is None or score > bestScore:
                bestScore = score
                bestM = m

            if bestScore != 0 and score < bestScore and score < 1 - target_error:
                break
        else:
            RMSE = math.sqrt(score[1])
            print("iteracija: " + str(counter) + ", Root Mean Square Error(RMSE): " + 
            str(round(RMSE,4)) +", size: "+ str(round(size,4)))
            if bestScore is None or RMSE < bestScore:
                bestScore = copy.deepcopy(RMSE)
                bestM = m

            if bestScore != 0 and RMSE > bestScore and RMSE > target_error:
                break
        
        size *= 0.9
        
        neighbors = getNeighbors(forest, forestMap, W)
        pruneTrees(neighbors, forest, int(len(W) * 0.1))
    return bestM, bestScore

## Klasifikacija MNIST seta 

In [29]:
#https://archive.ics.uci.edu/ml/datasets/covertype
#http://yann.lecun.com/exdb/mnist/

image_size = 28
train_data_count = 60_000
test_data_count = 10_000

def readGzImages(path,number_of_images):
    input_X = gzip.open(path,'r')
    input_X.read(16)
    buf_X = input_X.read(image_size * image_size * number_of_images)
    X = np.frombuffer(buf_X, dtype=np.uint8).astype(np.float32)
    X = X.reshape(number_of_images ,image_size * image_size)
    return X

def readGzLabels(path, number_of_labels):
    input_y = gzip.open(path,'r')
    input_y.read(8)
    buf_y = input_y.read(1 * number_of_labels)
    y = np.frombuffer(buf_y, dtype=np.uint8).astype(np.int32)
    y = y.reshape(number_of_labels)
    return y

X = readGzImages('./MNIST_set/train-images-idx3-ubyte.gz', train_data_count)
y = readGzLabels('./MNIST_set/train-labels-idx1-ubyte.gz', train_data_count)

X_test = readGzImages('./MNIST_set/t10k-images-idx3-ubyte.gz', test_data_count)
y_test = readGzLabels('./MNIST_set/t10k-labels-idx1-ubyte.gz', test_data_count)

In [30]:
_, score = refinedA(X,y,X_test, y_test,"classifation",iteration=10,)
print("Minimalni error: " + str(1 - score))


Accuracy = 97.55% (9755/10000) (classification)
iteracija: 0, error: 0.0245, len(W): 369784
Accuracy = 97.59% (9759/10000) (classification)
iteracija: 1, error: 0.0241, len(W): 332806
Accuracy = 97.63% (9763/10000) (classification)
iteracija: 2, error: 0.0237, len(W): 299526
Accuracy = 97.67% (9767/10000) (classification)
iteracija: 3, error: 0.0233, len(W): 269574
Accuracy = 97.76% (9776/10000) (classification)
iteracija: 4, error: 0.0224, len(W): 242617
Accuracy = 97.84% (9784/10000) (classification)
iteracija: 5, error: 0.0216, len(W): 218356
Accuracy = 97.88% (9788/10000) (classification)
iteracija: 6, error: 0.0212, len(W): 196521
Accuracy = 97.87% (9787/10000) (classification)
iteracija: 7, error: 0.0213, len(W): 176869
Accuracy = 97.85% (9785/10000) (classification)
iteracija: 8, error: 0.0215, len(W): 159183
Accuracy = 97.88% (9788/10000) (classification)
iteracija: 9, error: 0.0212, len(W): 143265
Accuracy = 97.9% (9790/10000) (classification)
iteracija: 10, error: 0.021, len(

In [31]:
_, score = refinedE(X,y,X_test, y_test,"classifation",target_error=0.0314)
print("Minimalni error: " + str(1 - score))

Accuracy = 97.74% (9774/10000) (classification)
iteracija: 0, error: 0.0226, size: 1
Accuracy = 97.8% (9780/10000) (classification)
iteracija: 1, error: 0.022, size: 0.9
Accuracy = 97.79% (9779/10000) (classification)
iteracija: 2, error: 0.0221, size: 0.81
Accuracy = 97.84% (9784/10000) (classification)
iteracija: 3, error: 0.0216, size: 0.729
Accuracy = 97.85% (9785/10000) (classification)
iteracija: 4, error: 0.0215, size: 0.6561
Accuracy = 97.82% (9782/10000) (classification)
iteracija: 5, error: 0.0218, size: 0.5905
Accuracy = 97.84% (9784/10000) (classification)
iteracija: 6, error: 0.0216, size: 0.5314
Accuracy = 97.76% (9776/10000) (classification)
iteracija: 7, error: 0.0224, size: 0.4783
Accuracy = 97.82% (9782/10000) (classification)
iteracija: 8, error: 0.0218, size: 0.4305
Accuracy = 97.8% (9780/10000) (classification)
iteracija: 9, error: 0.022, size: 0.3874
Accuracy = 97.89% (9789/10000) (classification)
iteracija: 10, error: 0.0211, size: 0.3487
Accuracy = 97.86% (9786/

## Klasifikacija Letter seta

In [85]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

#https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/

data = np.genfromtxt('./letter_set/letter-recognition.data', delimiter=',', dtype=np.str,encoding="utf8")
X = data[:,1:].astype('int32')
y = data[:,0]

le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)
X, X_test, y, y_test = train_test_split(X, y, test_size=0.25)

In [94]:
_, score = refinedA(X,y,X_test, y_test,"classifation",iteration=10)
print("Minimalni error: " + str(1 - score))


Accuracy = 96.48% (4824/5000) (classification)
iteracija: 1, error: 0.0352, len(W): 142914
Accuracy = 96.48% (4824/5000) (classification)
iteracija: 2, error: 0.0352, len(W): 128623
Accuracy = 96.56% (4828/5000) (classification)
iteracija: 3, error: 0.0344, len(W): 115761
Accuracy = 96.62% (4831/5000) (classification)
iteracija: 4, error: 0.0338, len(W): 104185
Accuracy = 96.56% (4828/5000) (classification)
iteracija: 5, error: 0.0344, len(W): 93767
Accuracy = 96.56% (4828/5000) (classification)
iteracija: 6, error: 0.0344, len(W): 84391
Accuracy = 96.7% (4835/5000) (classification)
iteracija: 7, error: 0.033, len(W): 75952
Accuracy = 96.72% (4836/5000) (classification)
iteracija: 8, error: 0.0328, len(W): 68357
Accuracy = 96.62% (4831/5000) (classification)
iteracija: 9, error: 0.0338, len(W): 61522
Accuracy = 96.72% (4836/5000) (classification)
iteracija: 10, error: 0.0328, len(W): 55370
Accuracy = 96.64% (4832/5000) (classification)
iteracija: 11, error: 0.0336, len(W): 49833
Minima

In [90]:
_, score = refinedE(X,y,X_test, y_test,"classifation",target_error=0.045)
print("Minimalni error: " + str(1 - score))

Accuracy = 96.58% (4829/5000) (classification)
iteracija: 1, error: 0.0342, size: 1
Accuracy = 96.64% (4832/5000) (classification)
iteracija: 2, error: 0.0336, size: 0.9
Accuracy = 96.66% (4833/5000) (classification)
iteracija: 3, error: 0.0334, size: 0.81
Accuracy = 96.72% (4836/5000) (classification)
iteracija: 4, error: 0.0328, size: 0.729
Accuracy = 96.64% (4832/5000) (classification)
iteracija: 5, error: 0.0336, size: 0.6561
Accuracy = 96.54% (4827/5000) (classification)
iteracija: 6, error: 0.0346, size: 0.5905
Accuracy = 96.52% (4826/5000) (classification)
iteracija: 7, error: 0.0348, size: 0.5314
Accuracy = 96.46% (4823/5000) (classification)
iteracija: 8, error: 0.0354, size: 0.4783
Accuracy = 96.52% (4826/5000) (classification)
iteracija: 9, error: 0.0348, size: 0.4305
Accuracy = 96.48% (4824/5000) (classification)
iteracija: 10, error: 0.0352, size: 0.3874
Accuracy = 96.4% (4820/5000) (classification)
iteracija: 11, error: 0.036, size: 0.3487
Accuracy = 96.4% (4820/5000) (cl

##  regresija CPUSMALL seta

In [56]:
# https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression.html
data = np.genfromtxt('./cpusmall/cpusmall_scale.txt', delimiter=',',encoding="utf8")
X = data[:,1:]
y = data[:,0]
X, X_test, y, y_test = train_test_split(X, y, test_size=0.4)

In [70]:
_, score = refinedA(X,y,X_test, y_test,"regression",iteration=20,c = 100, depth_size_index=1)
print("Najmanji RMSE: " + str(score))

Mean squared error = 8.40912 (regression)
Squared correlation coefficient = 0.974052 (regression)
iteracija: 1, Root Mean Square Error(RMSE): 2.8998, len(W): 85671
Mean squared error = 8.43188 (regression)
Squared correlation coefficient = 0.973912 (regression)
iteracija: 2, Root Mean Square Error(RMSE): 2.9038, len(W): 77104
Mean squared error = 8.48541 (regression)
Squared correlation coefficient = 0.973685 (regression)
iteracija: 3, Root Mean Square Error(RMSE): 2.913, len(W): 69394
Mean squared error = 8.50692 (regression)
Squared correlation coefficient = 0.973548 (regression)
iteracija: 4, Root Mean Square Error(RMSE): 2.9167, len(W): 62455
Mean squared error = 8.5319 (regression)
Squared correlation coefficient = 0.973457 (regression)
iteracija: 5, Root Mean Square Error(RMSE): 2.9209, len(W): 56210
Mean squared error = 8.69018 (regression)
Squared correlation coefficient = 0.972927 (regression)
iteracija: 6, Root Mean Square Error(RMSE): 2.9479, len(W): 50589
Mean squared error

In [72]:
_, score = refinedE(X,y,X_test, y_test,"regression",target_error=3.15,c = 100)
print("Najmanji RMSE: " + str(score))

Mean squared error = 8.54822 (regression)
Squared correlation coefficient = 0.973705 (regression)
iteracija: 1, Root Mean Square Error(RMSE): 2.9237, size: 1
Mean squared error = 8.45267 (regression)
Squared correlation coefficient = 0.973867 (regression)
iteracija: 2, Root Mean Square Error(RMSE): 2.9073, size: 0.9
Mean squared error = 8.50588 (regression)
Squared correlation coefficient = 0.973635 (regression)
iteracija: 3, Root Mean Square Error(RMSE): 2.9165, size: 0.81
Mean squared error = 8.46089 (regression)
Squared correlation coefficient = 0.973769 (regression)
iteracija: 4, Root Mean Square Error(RMSE): 2.9088, size: 0.729
Mean squared error = 8.35586 (regression)
Squared correlation coefficient = 0.974072 (regression)
iteracija: 5, Root Mean Square Error(RMSE): 2.8907, size: 0.6561
Mean squared error = 8.49914 (regression)
Squared correlation coefficient = 0.973557 (regression)
iteracija: 6, Root Mean Square Error(RMSE): 2.9153, size: 0.5905
Mean squared error = 8.62025 (reg

## Abalone reggresion set

In [73]:
# https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression.html
data = np.genfromtxt('./abalone_set/abalone.txt', delimiter=',',encoding="utf8")
X = data[:,1:]
y = data[:,0]
X, X_test, y, y_test = train_test_split(X, y, test_size=0.4)

In [81]:
_, score = refinedA(X,y,X_test, y_test,"regression",iteration=10,c = 100,depth_size_index=2)
print("Najmanji RMSE: " + str(score))

Mean squared error = 4.93249 (regression)
Squared correlation coefficient = 0.518492 (regression)
iteracija: 1, Root Mean Square Error(RMSE): 2.2209, len(W): 60715
Mean squared error = 4.94505 (regression)
Squared correlation coefficient = 0.517077 (regression)
iteracija: 2, Root Mean Square Error(RMSE): 2.2237, len(W): 54644
Mean squared error = 4.95536 (regression)
Squared correlation coefficient = 0.516264 (regression)
iteracija: 3, Root Mean Square Error(RMSE): 2.2261, len(W): 49180
Mean squared error = 4.98954 (regression)
Squared correlation coefficient = 0.513292 (regression)
iteracija: 4, Root Mean Square Error(RMSE): 2.2337, len(W): 44262
Mean squared error = 4.99935 (regression)
Squared correlation coefficient = 0.512582 (regression)
iteracija: 5, Root Mean Square Error(RMSE): 2.2359, len(W): 39836
Mean squared error = 5.00946 (regression)
Squared correlation coefficient = 0.5119 (regression)
iteracija: 6, Root Mean Square Error(RMSE): 2.2382, len(W): 35853
Mean squared error

In [84]:
_, score = refinedE(X,y,X_test, y_test,"regression",target_error=2.5,c = 10)
print("Najmanji RMSE: " + str(score))

Mean squared error = 5.10378 (regression)
Squared correlation coefficient = 0.503974 (regression)
iteracija: 1, Root Mean Square Error(RMSE): 2.2592, size: 1
Mean squared error = 5.12296 (regression)
Squared correlation coefficient = 0.501965 (regression)
iteracija: 2, Root Mean Square Error(RMSE): 2.2634, size: 0.9
Mean squared error = 5.15114 (regression)
Squared correlation coefficient = 0.498795 (regression)
iteracija: 3, Root Mean Square Error(RMSE): 2.2696, size: 0.81
Mean squared error = 5.16948 (regression)
Squared correlation coefficient = 0.497101 (regression)
iteracija: 4, Root Mean Square Error(RMSE): 2.2736, size: 0.729
Mean squared error = 5.21666 (regression)
Squared correlation coefficient = 0.493038 (regression)
iteracija: 5, Root Mean Square Error(RMSE): 2.284, size: 0.6561
Mean squared error = 5.27526 (regression)
Squared correlation coefficient = 0.488158 (regression)
iteracija: 6, Root Mean Square Error(RMSE): 2.2968, size: 0.5905
Mean squared error = 5.29792 (regr


# Drugi dio projekta
### url članka: https://www.jmlr.org/papers/volume22/20-1031/20-1031.pdf
u članku je opisano korištenje SNN neuronskih mreža za klasifikaciju, te je algoritam pokazan na različitim MNIST setovima. 
Korišteni datasetovi: MNIST, EMNIST, Fashion-MNIST. neuromorphic_MNIST nije primjenjiv na ovoj algoritam jer se radi o videozapisima


## Klasifikacija EMNIST seta

In [9]:
#https://www.kaggle.com/crawford/emnist?select=emnist-balanced-mapping.txt
data_train = np.genfromtxt('./EMINIST/emnist-balanced-train.csv', delimiter=',',encoding="utf8")
X_train = data_train[:,1:]
y_train = data_train[:,0]
data_test = np.genfromtxt('./EMINIST/emnist-balanced-test.csv', delimiter=',',encoding="utf8")
X_test = data_test[:,1:]
y_test = data_test[:,0]

In [10]:
_, score = refinedA(X_train,y_train,X_test, y_test,"classifation",iteration=10)
print("Minimalni error: " + str(1 - score))

Accuracy = 82.5053% (15511/18800) (classification)
iteracija: 1, error: 0.1749, len(W): 1468381
Accuracy = 82.5851% (15526/18800) (classification)
iteracija: 2, error: 0.1741, len(W): 1321543
Accuracy = 82.6489% (15538/18800) (classification)
iteracija: 3, error: 0.1735, len(W): 1189389
Accuracy = 82.8245% (15571/18800) (classification)
iteracija: 4, error: 0.1718, len(W): 1070451
Accuracy = 83.1011% (15623/18800) (classification)
iteracija: 5, error: 0.169, len(W): 963406
Accuracy = 82.9149% (15588/18800) (classification)
iteracija: 6, error: 0.1709, len(W): 867066
Accuracy = 83.0532% (15614/18800) (classification)
iteracija: 7, error: 0.1695, len(W): 780360
Accuracy = 83% (15604/18800) (classification)
iteracija: 8, error: 0.17, len(W): 702324
Accuracy = 83.133% (15629/18800) (classification)
iteracija: 9, error: 0.1687, len(W): 632092
Accuracy = 83.1436% (15631/18800) (classification)
iteracija: 10, error: 0.1686, len(W): 568883
Accuracy = 83.0638% (15616/18800) (classification)
ite

In [11]:
_, score = refinedE(X_train,y_train,X_test, y_test,"classifation",target_error=0.175)
print("Minimalni error: " + str(1 - score))

Accuracy = 82.7181% (15551/18800) (classification)
iteracija: 1, error: 0.1728, size: 1
Accuracy = 82.6543% (15539/18800) (classification)
iteracija: 2, error: 0.1735, size: 0.9
Accuracy = 82.7128% (15550/18800) (classification)
iteracija: 3, error: 0.1729, size: 0.81
Accuracy = 82.75% (15557/18800) (classification)
iteracija: 4, error: 0.1725, size: 0.729
Accuracy = 82.9309% (15591/18800) (classification)
iteracija: 5, error: 0.1707, size: 0.6561
Accuracy = 82.8085% (15568/18800) (classification)
iteracija: 6, error: 0.1719, size: 0.5905
Accuracy = 82.9574% (15596/18800) (classification)
iteracija: 7, error: 0.1704, size: 0.5314
Accuracy = 82.9149% (15588/18800) (classification)
iteracija: 8, error: 0.1709, size: 0.4783
Accuracy = 83.0106% (15606/18800) (classification)
iteracija: 9, error: 0.1699, size: 0.4305
Accuracy = 83.0904% (15621/18800) (classification)
iteracija: 10, error: 0.1691, size: 0.3874
Accuracy = 83.0798% (15619/18800) (classification)
iteracija: 11, error: 0.1692, s

## Klasifikacija Fashion-MNIST seta

In [103]:
#https://www.kaggle.com/crawford/emnist?select=emnist-balanced-mapping.txt
data_train = np.genfromtxt('./Fashion-MNIST/fashion-mnist_train.csv', delimiter=',',encoding="utf8")
X_train = data_train[1:,1:]
y_train = data_train[1:,0]
data_test = np.genfromtxt('./Fashion-MNIST/fashion-mnist_test.csv', delimiter=',',encoding="utf8")
X_test = data_test[1:,1:]
y_test = data_test[1:,0]

In [105]:
_, score = refinedA(X_train,y_train,X_test, y_test,"classifation",iteration=10)
print("Minimalni error: " + str(1 - score))

Accuracy = 89.72% (8972/10000) (classification)
iteracija: 1, error: 0.1028, len(W): 375195
Accuracy = 89.76% (8976/10000) (classification)
iteracija: 2, error: 0.1024, len(W): 337676
Accuracy = 89.74% (8974/10000) (classification)
iteracija: 3, error: 0.1026, len(W): 303909
Accuracy = 89.77% (8977/10000) (classification)
iteracija: 4, error: 0.1023, len(W): 273519
Accuracy = 89.71% (8971/10000) (classification)
iteracija: 5, error: 0.1029, len(W): 246168
Accuracy = 89.82% (8982/10000) (classification)
iteracija: 6, error: 0.1018, len(W): 221552
Accuracy = 89.82% (8982/10000) (classification)
iteracija: 7, error: 0.1018, len(W): 199397
Accuracy = 89.76% (8976/10000) (classification)
iteracija: 8, error: 0.1024, len(W): 179458
Accuracy = 89.84% (8984/10000) (classification)
iteracija: 9, error: 0.1016, len(W): 161513
Accuracy = 89.59% (8959/10000) (classification)
iteracija: 10, error: 0.1041, len(W): 145362
Accuracy = 89.61% (8961/10000) (classification)
iteracija: 11, error: 0.1039, l

In [106]:
_, score = refinedE(X_train,y_train,X_test, y_test,"classifation",target_error=0.11)
print("Minimalni error: " + str(1 - score))

Accuracy = 89.64% (8964/10000) (classification)
iteracija: 1, error: 0.1036, size: 1
Accuracy = 89.77% (8977/10000) (classification)
iteracija: 2, error: 0.1023, size: 0.9
Accuracy = 89.82% (8982/10000) (classification)
iteracija: 3, error: 0.1018, size: 0.81
Accuracy = 89.91% (8991/10000) (classification)
iteracija: 4, error: 0.1009, size: 0.729
Accuracy = 89.9% (8990/10000) (classification)
iteracija: 5, error: 0.101, size: 0.6561
Accuracy = 89.96% (8996/10000) (classification)
iteracija: 6, error: 0.1004, size: 0.5905
Accuracy = 89.8% (8980/10000) (classification)
iteracija: 7, error: 0.102, size: 0.5314
Accuracy = 89.76% (8976/10000) (classification)
iteracija: 8, error: 0.1024, size: 0.4783
Accuracy = 89.79% (8979/10000) (classification)
iteracija: 9, error: 0.1021, size: 0.4305
Accuracy = 89.69% (8969/10000) (classification)
iteracija: 10, error: 0.1031, size: 0.3874
Accuracy = 89.56% (8956/10000) (classification)
iteracija: 11, error: 0.1044, size: 0.3487
Accuracy = 89.48% (8948